In [2]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster
from geopy.distance import geodesic
import ipywidgets as widgets
from IPython.display import display, clear_output

# Charger les données
df = pd.read_csv('df_fournisseurs.csv', encoding='utf-8-sig')
df.columns = df.columns.str.strip()

# Renommer les colonnes
df = df.rename(columns={
    'Société': 'Societe',
    'Biostimulants & Growth Enhancers': 'Biostimulants',
    'Fertilizers & Soil Conditioners': 'Fertilizers',
    'Microbial & Organic Solutions': 'Microbial',
    'Pesticides & Fungicides': 'Pesticides',
    'General Agriculture & Other': 'General_Agriculture'
})

# Nettoyage
df['Latitude'] = pd.to_numeric(df['Latitude'], errors='coerce')
df['Longitude'] = pd.to_numeric(df['Longitude'], errors='coerce')
df = df.dropna(subset=['Latitude', 'Longitude'])

# Fonction pour la distance
def calculate_distance(lat1, lon1, lat2, lon2):
    return geodesic((lat1, lon1), (lat2, lon2)).km

# Widgets UI
latitude_input = widgets.FloatText(value=36.8, description='Latitude:', step=0.0001)
longitude_input = widgets.FloatText(value=10.1, description='Longitude:', step=0.0001)
rayon_input = widgets.FloatSlider(value=50, min=1, max=500, step=1, description='Rayon (km):', style={'description_width': 'initial'})
search_button = widgets.Button(description="🔍 Trouver", button_style='success')
title_label = widgets.HTML(value="<h3>🌍 Recherche de fournisseurs autour d'une position</h3>")
output = widgets.Output()

def on_button_clicked(b):
    with output:
        clear_output()
        user_lat = latitude_input.value
        user_lon = longitude_input.value
        rayon = rayon_input.value
        
        df['Distance'] = df.apply(lambda row: calculate_distance(user_lat, user_lon, row['Latitude'], row['Longitude']), axis=1)
        nearby = df[df['Distance'] <= rayon].sort_values('Distance')
        
        # Créer la carte
        m = folium.Map(location=[user_lat, user_lon], zoom_start=10)
        folium.Marker(
            [user_lat, user_lon],
            popup="Votre position",
            icon=folium.Icon(color='red', icon='user')
        ).add_to(m)
        
        marker_cluster = MarkerCluster().add_to(m)
        
        for _, row in nearby.iterrows():
            product_types = {
                'Biostimulants': row['Biostimulants'],
                'Fertilizers': row['Fertilizers'],
                'Microbial': row['Microbial'],
                'Pesticides': row['Pesticides'],
                'Other': row['Other'],
                'General_Agriculture': row['General_Agriculture']
            }
            main_type = max(product_types, key=product_types.get)
            color_map = {
                'Biostimulants': 'green',
                'Fertilizers': 'blue',
                'Microbial': 'purple',
                'Pesticides': 'orange',
                'Other': 'gray',
                'General_Agriculture': 'beige'
            }
            icon_color = color_map.get(main_type, 'gray')
            products = row['Produits'].split() if isinstance(row['Produits'], str) else []
            products_list = "<br>".join([f"• {p}" for p in products[:5]])
            if len(products) > 5:
                products_list += "<br>• ..."
            popup_content = f"""
            <div style="width:480px; font-size:14px;">
                <b>{row['Societe']}</b><br>
                <i>{row['Adresse']}</i><br><br>
                <b>Distance:</b> {row['Distance']:.1f} km<br>
                <b>Type principal:</b> {main_type}<br>
                <b>Produits:</b><br>
                {products_list}
            </div>
            """
            folium.Marker(
                [row['Latitude'], row['Longitude']],
                popup=folium.Popup(popup_content, max_width=500),
                icon=folium.Icon(color=icon_color, icon='industry')
            ).add_to(marker_cluster)
        
        display(m)
        print(f"\nFournisseurs dans un rayon de {rayon} km :")
        display(nearby[['Societe', 'Adresse', 'Distance', 'Produits']])

# Lier bouton
search_button.on_click(on_button_clicked)

# Afficher l'interface utilisateur
ui = widgets.VBox([
    title_label,
    widgets.HBox([latitude_input, longitude_input]),
    rayon_input,
    search_button,
    output
])

display(ui)


In [2]:
!pip install geopy

  Obtaining dependency information for geopy from https://files.pythonhosted.org/packages/e5/15/cf2a69ade4b194aa524ac75112d5caac37414b20a3a03e6865dfe0bd1539/geopy-2.4.1-py3-none-any.whl.metadata
  Obtaining dependency information for geographiclib<3,>=1.52 from https://files.pythonhosted.org/packages/9f/5a/a26132406f1f40cf51ea349a5f11b0a46cec02a2031ff82e391c2537247a/geographiclib-2.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/125.4 kB ? eta -:--:--
   --- ------------------------------------ 10.2/125.4 kB ? eta -:--:--
   ------------ -------------------------- 41.0/125.4 kB 653.6 kB/s eta 0:00:01
   ---------------------------------------- 125.4/125.4 kB 1.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/40.3 kB ? eta -:--:--
   ---------------------------------------- 40.3/40.3 kB 2.0 MB/s eta 0:00:00
